In [ ]:
# import torch
# num_of_gpus = torch.cuda.device_count()
# print(num_of_gpus)

In [ ]:
# import torch

# device = torch.device("cuda:0")
# device_properties = torch.cuda.get_device_properties(device)

# print(f"Total memory available: {device_properties.total_memory / 1024**2:.2f} MB")

In [ ]:
# # initialize a tensor on the GPU
# x = torch.randn(1000, 1000).cuda()

# # measure the amount of memory used by the tensor
# memory_used = torch.cuda.max_memory_allocated() / 1024**2  # convert to MB

# print(f"Memory used: {memory_used:.2f} MB")

In [ ]:
# !pip install transformers
# !pip install datasets
# !pip install umap-learn
# !pip install sentencepiece

In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
from transformers import AutoModelForSequenceClassification
from datasets import load_dataset
from transformers import AutoTokenizer, AutoConfig
from transformers import AutoModel
from torch import torch
import pandas as pd
import matplotlib.pyplot as plt

from datasets import Dataset, DatasetDict
from datasets import ClassLabel, concatenate_datasets
from datasets import Dataset, ClassLabel, Features, Value
from sklearn.model_selection import train_test_split

import datasets
import pandas as pd

# para construir gráficas y realizar análisis exploratorio de los datos
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px

# para cargar datos y realizar pre-procesamiento básico
from collections import Counter

# Cargamos esta librería para revisar el desbalance que existe en el dataset de tripadvisor
from imblearn.under_sampling import RandomUnderSampler

# CARGANDO DATOS DESDE DIFERENTES ORIGENES DE DATOS
Aquí cargamos datos de diferentes lugare
1. CARDIFF-TWEETS: Cargamos tweets desde la base de datos de cardiff: https://huggingface.co/datasets/cardiffnlp/tweet_sentiment_multilingual
2. DA-VINCIS-2022: Cargamos los datos del concurso del 2022. 
3. DA-VINCIS-2023: Cargamos los datos del concurso del 2023.

In [ ]:
TEXT_COL      = 'text'
CLASS_COL_ST1 = 'label'
IMAGES = 'images'

## CARDIFF-TWEETS
Datos del dataset de Cardiff Multilingual Tweets, de este dataset solo se extrajeron los tweets en español.

In [ ]:
# extended_df = load_dataset("cardiffnlp/tweet_sentiment_multilingual", 'spanish')
# descomente la siguiente línea para leer datos desde un URL

extended_train_path = 'https://raw.githubusercontent.com/carlossuazo/davincis-iberlef-2023/main/data/cardiff-tweets-db/extended_train.csv'#'data/extended/extended_train.csv'
extended_validation_path = 'https://raw.githubusercontent.com/carlossuazo/davincis-iberlef-2023/main/data/cardiff-tweets-db/extended_validation.csv'#'data/extended/extended_validation.csv'
extended_test_path = 'https://raw.githubusercontent.com/carlossuazo/davincis-iberlef-2023/main/data/cardiff-tweets-db/extended_test.csv'#'data/extended/extended_test.csv'

# Reading the data
df_extended_train = pd.read_csv(extended_train_path)
df_extended_validation = pd.read_csv(extended_validation_path)
df_extended_test = pd.read_csv(extended_test_path)


display(df_extended_train.shape)
display(df_extended_validation.shape)
display(df_extended_test.shape)

In [ ]:
df_extended_train["label"] = df_extended_train["label"].apply(lambda x: 1 if x == 0 else 0)
df_extended_validation["label"] = df_extended_validation["label"].apply(lambda x: 1 if x == 0 else 0)
df_extended_test["label"] = df_extended_test["label"].apply(lambda x: 1 if x == 0 else 0)

In [ ]:
display(df_extended_train[:12])
display(df_extended_validation[:12])
display(df_extended_test[:12])

In [ ]:
df_extended = pd.concat([df_extended_train, df_extended_validation, df_extended_test])

### Undersampling de Dataset 
Debido a que la base de datos de Cardiff está etiquetada para hacer análisis de sentimiento obre tres clases [Neutral, Positivo, Negativo], el dataset los hemos nivelado para que neutral y positivo sean una sola clase, por lo que nos queda una dataset desnivelado.

In [ ]:
# obtener algunas estadísticas sobre los datos
categories = sorted(df_extended[CLASS_COL_ST1].unique(), reverse=False)
hist= Counter(df_extended[CLASS_COL_ST1]) 
print(f'Total de instancias -> {df_extended.shape[0]}')
print(f'Distribución de clases -> {{item[0]:round(item[1]/len(df_extended[CLASS_COL_ST1]), 3) for item in sorted(hist.items(), key=lambda x: x[0])}}')

print(f'Categorías -> {categories}')
print(f'Comentario de ejemplo -> {df_extended[TEXT_COL][0]}')
print(f'Categoría del comentario -> {df_extended[CLASS_COL_ST1][0]}')

fig = go.Figure(layout=go.Layout(height=400, width=600))
fig.add_trace(go.Bar(x=categories, y=[hist[cat] for cat in categories]))
fig.show()

print('Done!')

In [ ]:
print(df_extended.head())

X = df_extended.drop('label', axis=1)
y = df_extended['label']
# resumen de las clases para ver como se encuentran distribuidas
print(Counter(y))
# definimos las estrategia de undersampling, en nuestro caso buscamos que todas
# clases se nivelen a la clase minoritaria.
undersample = RandomUnderSampler(sampling_strategy='not minority', random_state = 0)
# hacemos el fit de nuestro conjunto de datos a nivelar
X_over, y_over = undersample.fit_resample(X, y)
print(Counter(y_over))

df_extended = pd.concat([X_over, y_over], axis=1)
print(df_extended.head())

In [ ]:
df_extended_train, df_extended_test = train_test_split(df_extended, test_size=0.15)
df_extended_train, df_extended_validation = train_test_split(df_extended_train, test_size=0.15)

In [ ]:
df_extended_train.shape, df_extended_test.shape, df_extended_validation.shape 

In [ ]:
# for split, data in extended_df.items():
#     data.to_csv(f"/extended_{split}.csv")

In [ ]:
# display(extended_df)
# display(extended_df['train'].features)

In [ ]:
# extended_df['train'].features

In [ ]:
# # Create a ClassLabel object
# class_label = ClassLabel(names=['no', 'yes'])

# # Convert a class name to a class index
# class_index = class_label.str2int('yes')
# print(class_index) # 1

# # Convert a class index to a class name
# class_name = class_label.int2str(0)
# print(class_name) # 'no'

In [ ]:

# # Define a function that takes in a row and returns the updated row
# def update_row(example):
#     # Get the class name for the current class index
#     class_name = extended_df['train'].features['label'].int2str(example['label'])
    
#     # Convert the class name to the new class index
#     new_class_index = class_label.str2int(class_name)
    
#     # Update the value of the class label column
#     example['label'] = new_class_index
#     return example

# # Update the dataset using the update_row function
# updated_dataset = extended_df.map(update_row)

# # Update the dataset's features to use the new ClassLabel object
# updated_features = updated_dataset.features.copy()
# updated_features['label'] = class_label
# updated_dataset = Dataset.from_dict(updated_dataset.to_dict(), features=updated_features)

## DA-VINCIS DATA 2022
Obtenemos los datos del concurso 2022

In [ ]:
train_url = 'https://raw.githubusercontent.com/carlossuazo/davincis-iberlef-2023/main/data/data-davincis-2022/training_data/train.csv'
test_url = 'https://raw.githubusercontent.com/carlossuazo/davincis-iberlef-2023/main/data/data-davincis-2022/test_data/test.csv'

# Dataset de entrenamiento
df_train_22 = pd.read_csv(train_url, header=None, names = [TEXT_COL, CLASS_COL_ST1])

# Dataset de test
df_test_22  = pd.read_csv(test_url, header=None, names = [TEXT_COL, CLASS_COL_ST1])

display(df_train_22)
display(df_test_22)

In [ ]:
df_22 = pd.concat([df_train_22, df_test_22])
display(df_22[:12])

In [ ]:
df_train_22, df_test_22 = train_test_split(df_22, test_size=0.15, stratify = df_22[CLASS_COL_ST1])
df_train_22, df_validation_22 = train_test_split(df_train_22, test_size=0.15, stratify = df_train_22[CLASS_COL_ST1])

## DA-VINCIS DATA 2023

In [ ]:
train_url_23 = 'https://raw.githubusercontent.com/carlossuazo/davincis-iberlef-2023/main/data/data-davincis-2023/development_data_davincis23/train_data.csv'
train_labels_url_23 = 'https://raw.githubusercontent.com/carlossuazo/davincis-iberlef-2023/main/data/data-davincis-2023/development_data_davincis23/train_labels_subtask_1.csv'

test_url_23 = 'https://raw.githubusercontent.com/carlossuazo/davincis-iberlef-2023/main/data/data-davincis-2023/test_data/test_data.csv'
validation_url_23 = 'https://raw.githubusercontent.com/carlossuazo/davincis-iberlef-2023/main/data/data-davincis-2023/development_data_davincis23/validation_data.csv'

# Dataset de entrenamiento
df_train_23 = pd.read_csv(train_url_23, header=None, names = [IMAGES,TEXT_COL], encoding='latin1')
df_train_labels_23 = pd.read_csv(train_labels_url_23, header=None, names = [CLASS_COL_ST1])

# Dataset de test
df_test_23 = pd.read_csv(test_url, header=None, names = [IMAGES,TEXT_COL], encoding='latin1')
df_validation_23 = pd.read_csv(validation_url_23, header=None, names = [IMAGES,TEXT_COL], encoding='latin1')

df_train_23 = df_train_23.drop('images', axis=1)

# Estos son los conjuntos de entrenamiento y validación que necesitamos para validar el conjunto de datos de entrenamiento
# que se subiran a la competencia
_df_test__23 = df_test_23.drop('images', axis=1)
_df_validation__23 = df_validation_23.drop('images', axis=1)

display(df_train_23[:12])
display(_df_test__23[:12])
display(df_train_labels_23[:12])
display(_df_validation__23[:12])

In [ ]:
df_train_23[CLASS_COL_ST1] = df_train_labels_23[CLASS_COL_ST1]
print(df_train_23[:12])
df_23 = df_train_23

In [ ]:
df_train_23, df_test_23 = train_test_split(df_23, test_size=0.15, stratify = df_23[CLASS_COL_ST1])
df_train_23, df_validation_23 = train_test_split(df_train_23, test_size=0.15, stratify = df_train_23[CLASS_COL_ST1])

In [ ]:
df_train = pd.concat([df_train_23, df_extended_train], ignore_index=True)
df_validation = pd.concat([df_validation, df_extended_train], ignore_index=True)
df_test = pd.concat([df_test, df_extended_test], ignore_index=True)

# df_train["label"] = df_train["label"].apply(lambda x: 'harmfull' if x == 1 else 'no_harmfull')
# df_validation["label"] = df_validation["label"].apply(lambda x: 'harmfull' if x == 1 else 'no_harmfull')
# df_test["label"] = df_test["label"].apply(lambda x: 'harmfull' if x == 1 else 'no_harmfull')

display(df_train)
display(df_validation)
display(df_test)

In [ ]:
df_train = df_train.reset_index(drop=True)
df_validation = df_validation.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [ ]:
# Display the original column names
print("Original column names:")
print(df_train.columns)

# Display the original column names
print("Original column names:")
print(df_test.columns)

# Display the original column names
print("Original column names:")
print(df_validation.columns)


In [ ]:
df = DatasetDict({
    "train": Dataset.from_pandas(df_train, features=Features({'text': Value('string'),'label': ClassLabel(names=['no_harmfull', 'harmfull'])})),
    "test": Dataset.from_pandas(df_test, features=Features({'text': Value('string'),'label': ClassLabel(names=['no_harmfull', 'harmfull'])})),
    "validation": Dataset.from_pandas(df_validation, features=Features({'text': Value('string'),'label': ClassLabel(names=['no_harmfull', 'harmfull'])}))
    })

In [ ]:
df['train'].features


In [ ]:
display(df['train'][15:20])

In [ ]:
df

In [ ]:
df['train'].features

In [ ]:
model_ckpt = f'cardiffnlp/twitter-xlm-roberta-base-sentiment'#"distilbert-base-uncased-finetuned-sst-2-english" #"cardiffnlp/twitter-roberta-base-sentiment"#"cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
text = 'Tokenizar el texto es una tarea central del NLP.'
encoded_text = tokenizer(text)
print(encoded_text)

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens)

In [ ]:
print(tokenizer.convert_tokens_to_string(tokens))

In [ ]:
tokenizer.vocab_size

In [ ]:
tokenizer.model_max_length

In [ ]:
tokenizer.model_input_names

In [ ]:
def tokenize(batch):
    return tokenizer(batch[TEXT_COL], padding=True, truncation=True)

In [ ]:
print(tokenize(df["train"][:2]))

In [ ]:
tweets_encoded = df.map(tokenize, batched=True, batch_size=None) #batch_size=None

In [ ]:
# al aplicar map se agregan dos nuevas columnas al objeto, 
print(tweets_encoded["train"].column_names)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

In [ ]:
if device.type == "cuda":
    print("Device is a GPU")
else:
    print("Device is a CPU")

In [ ]:
from transformers import TFAutoModel

#tf_model = TFAutoModel.from_pretrained(model_ckpt)

In [ ]:
#tf_xlmr = TFAutoModel.from_pretrained("xlm-roberta-base")

In [ ]:
#tf_xlmr = TFAutoModel.from_pretrained("xlm-roberta-base", from_pt=True)

In [ ]:
text = "this is a test"
inputs = tokenizer(text, return_tensors="pt")
print(f"Input tensor shape: {inputs['input_ids'].size()}")

In [ ]:
inputs = {k:v.to(device) for k,v in inputs.items()}
with torch.no_grad():
    outputs = model(**inputs)
print(outputs)

In [ ]:
outputs.last_hidden_state.size()

In [ ]:
outputs.last_hidden_state[:,0].size()

In [ ]:
def extract_hidden_states(batch):
    # Place model inputs on the GPU
    inputs = {k:v.to(device) for k,v in batch.items()
              if k in tokenizer.model_input_names}
    # Extract last hidden states
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
    # Return vector for [CLS] token
    return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}
# def extract_hidden_states(batch):
#     # Place model inputs on the GPU
#     inputs = {k: torch.tensor(v).to(device) if isinstance(v, list) else v.to(device)
#               for k,v in batch.items()
#               if k in tokenizer.model_input_names}
#     # Extract last hidden states
#     with torch.no_grad():
#         last_hidden_state = model(**inputs).last_hidden_state
#     # Return vector for [CLS] token
#     return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}

In [ ]:
tweets_encoded.set_format("torch",  columns=["input_ids", "attention_mask", CLASS_COL_ST1])

In [ ]:
tweets_hidden = tweets_encoded.map(extract_hidden_states, batched=True)

In [ ]:
def label_int2str(row):
    return df["train"].features[CLASS_COL_ST1].int2str(row)

In [ ]:
tweets_hidden["train"].column_names

In [ ]:
import numpy as np

X_train = np.array(tweets_hidden["train"]["hidden_state"])
y_train = np.array(tweets_hidden["train"][CLASS_COL_ST1])

X_test = np.array(tweets_hidden["test"]["hidden_state"])
y_test = np.array(tweets_hidden["test"][CLASS_COL_ST1])

X_val = np.array(tweets_hidden["validation"]["hidden_state"])
y_val = np.array(tweets_hidden["validation"][CLASS_COL_ST1])

X_train.shape, X_test.shape

In [ ]:
from umap import UMAP
from sklearn.preprocessing import MinMaxScaler

# Scale features to [0,1] range
X_scaled = MinMaxScaler().fit_transform(X_train)
# Initialize and fit UMAP
mapper = UMAP(n_components=2, metric="cosine").fit(X_scaled)
# Create a DataFrame of 2D embeddings
df_emb = pd.DataFrame(mapper.embedding_, columns=["X", "Y"])
df_emb[CLASS_COL_ST1] = y_train
df_emb.head()

In [ ]:
df['train'].features

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(7,5))
axes = axes.flatten()
cmaps = ["Greys", "Blues", "Oranges", "Reds", "Purples", "Greens"]
labels = df["train"].features["label"].names

for i, (label, cmap) in enumerate(zip(labels, cmaps)):
    df_emb_sub = df_emb.query(f"label == {i}")
    axes[i].hexbin(df_emb_sub["X"], df_emb_sub["Y"],# cmap=cmap,
                   gridsize=20, linewidths=(0,))
    axes[i].set_title(label)
    axes[i].set_xticks([]), axes[i].set_yticks([])

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression

# We increase `max_iter` to guarantee convergence
lr_clf = LogisticRegression(max_iter=5000)
lr_clf.fit(X_train, y_train)
lr_clf.score(X_val, y_val)

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
dummy_clf.score(X_val, y_val)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    fig, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    plt.title("Normalized confusion matrix")
    plt.show()

y_preds = lr_clf.predict(X_val)
plot_confusion_matrix(y_preds, y_val, labels)

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 2
model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt, num_labels=num_labels, ignore_mismatched_sizes=True)
         .to(device))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 10
logging_steps = len(tweets_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=10,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  #push_to_hub=True,
                                  log_level="error")

In [ ]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=tweets_encoded["train"],
                  eval_dataset=tweets_encoded["test"],
                  tokenizer=tokenizer)
trainer.train();

In [ ]:
preds_output = trainer.predict(tweets_encoded["validation"])

In [ ]:
preds_output.metrics

In [ ]:
y_preds = np.argmax(preds_output.predictions, axis=1)

In [ ]:
plot_confusion_matrix(y_preds, y_val, labels)